### 1. Stemming words 

Stemming is a technique to remove affixes from a word, ending up with a stem.  Stemming is used by search enginers for indexing words.  Instead of storing all forms of a word, a search engine can store only the stems, greatly reducing the size of the index while increasing retrieval accuracy.  In NLTK, the most common stemming algorithm is the Porter stemming algorithm: 

In [1]:
# Porter stemming algorithm 
from nltk.stem import PorterStemmer 
stemmer = PorterStemmer()
stemmer.stem('cooking')

'cook'

There are other stemmers available on NLTK, but there is no clear benefit of one vs. another.  

### 2. Lemmatizing words with WordNet 

Lemmatization is similar to stemming, but is more akin to synonym replacement.  A lemma is a root word, as opposed to the root stem.  Unlike stemming, we are always left with a valid word that means essentially the same thing.  However, the word may be completely different.  Here is an example: 

In [2]:
# Lemmatization example 
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('cooking')

'cooking'

In [3]:
lemmatizer.lemmatize('cooking', pos='v')

'cook'

As we can see, specifying the part of speech with lemmatization is important.  Here's another example of stemming vs. lemmatization: 

In [4]:
# Stemming vs. lemmatization 
from nltk.stem import PorterStemmer 
stemmer = PorterStemmer()
print(stemmer.stem('believes'))
print(lemmatizer.lemmatize('believes'))

believ
belief


Instead of chopping off the "es" like the `PorterStemmer` class, the `WordNetLemmatizer` class finds a valid root word.  Where a stemmer only looks at the form of the word, the lemmatizer looks at the meaning of the word.  By returning a lemma, we always get a valid word.  

### 3. Replacing words matching regular expressions 

Before, when we used tokenization on contractions, the tokenizers ran into trouble.  This aims to replace contractions with their expanded form.  Here's how we can set it up.  We define a number of replacement patters, then create a `RegexpReplacer` class to compile the patterns and provide a `replace()` method to substitute all the found patterns with their replacements: 

In [5]:
# Use regular expressions to replace patterns 
import re 

replacements = [
    (r'won\'t', 'will not'),
    (r'can\'t', 'cannot'),
    (r'i\'m', 'i am'),
    (r'ain\'t', 'is not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)n\'t', '\g<1> not'),
    (r'(\w+)\'ve', '\g<1> have'),
    (r'(\w+)\'s', '\g<1> is'),
    (r'(\w+)\'re', '\g<1> are'),
    (r'(\w+)\'d', '\g<1> would')
]

class RegexpReplacer(object):
    def __init__(self, patterns=replacements):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
        
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns: 
            s = re.sub(pattern, repl, s)
        return s

In [6]:
replacer = RegexpReplacer()
replacer.replace("can't is a contraction")

'cannot is a contraction'

In [7]:
replacer.replace("I should've done that thing I didn't do")

'I should have done that thing I did not do'

In [8]:
replacer.replace("I ain't gonna do that.  Didn't I tell you?")

'I is not gonna do that.  Did not I tell you?'

We can use this class as a preliminary step before tokenization and include it in a more convenient form: 

In [9]:
# Include replacement before tokenization
from nltk.tokenize import word_tokenize 
word_tokenize(replacer.replace("can't is a contraction"))

['can', 'not', 'is', 'a', 'contraction']

### 4. Removing repeating characters 

Text can be annoying to deal with when we encounter repeating characters, such as "I loooooove it", so we need a way to deal with them.  We can create a class that has the same form as the `RegexpReplacer` to take a single word and return a form without the repeating characters.  

In [10]:
# Repeating character replacer 
import re 

class RepeatReplacer(object):
    def __init__(self, word=None):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
        
    def replace(self, word):
        repl_word = self.repeat_regexp.sub(self.repl, word)
        
        if repl_word != word: 
            return self.replace(repl_word)
        else: 
            return repl_word

In [11]:
replacer = RepeatReplacer()
replacer.replace('loooooooove')

'love'

In [12]:
replacer.replace('ooooooooooh')

'oh'

However, this isn't perfect, because if we use any woord with two repeating characters, it will shorten it as well: 

In [13]:
replacer.replace('goose')

'gose'

So to fix this, let's create a new class that augments the `replace()` function with a WordNet lookup: 

In [14]:
# Incorporate WordNet lookup 
import re 
from nltk.corpus import wordnet 

class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
        
    def replace(self, word):
        if wordnet.synsets(word):
            return word 
        repl_word = self.repeat_regexp.sub(self.repl, word)
        
        if repl_word != word: 
            return self.replace(repl_word)
        else: 
            return repl_word

In [15]:
replacer = RepeatReplacer()
replacer.replace('goose')

'goose'

### 5. Spelling corrections 

Correcting spelling errors is another common problem encountered.  We can use the `Enchant` library for help with this.  

In [16]:
# Spelling correction 
import enchant 
from nltk.metrics import edit_distance 

class SpellingReplacer(object):
    def __init__(self, dict_name='en', max_dist=2):
        self.spell_dict = enchant.Dict(dict_name)
        self.max_dist = max_dist 
        
    def replace(self, word):
        if self.spell_dict.check(word):
            return word
        suggestions = self.spell_dict.suggest(word)
        
        if suggestions and edit_distance(word, suggestions[0]) <= self.max_dist:
            return suggestions[0]
        else: 
            return word 

ImportError: The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details
